# Restricted Boltzmann Machine
Dataset and code by Giacomo Torlai, Juan Carrasquilla and Lauren Hayward Sierens

`Goal`: This code will train a Restricted Boltzmann Machine (RBM) to learn the
distribution of spin configurations of the 2D Ising model at a
given temperature. After training, we will generate states from this trained model. Finally, we'll use these new states to calculate energy, magnetization, specific heat and susceptibility and compare them to those in Monte Carlo result. 

`What's given`:  20000 states for each of 11 different temperatures (1.0, 1.254, 1.508, 1.762, 2.016, 2.269, 2.524, 2.778, 3.032, 3.286 and 3.54) generated by MCMC from Boltzmann distribution.

In [1]:
from __future__ import print_function
import tensorflow as tf
import itertools as it
from rbm import RBM
import matplotlib.pyplot as plt
import numpy as np
import math
import os

In [2]:
#Specify font sizes for plots:
plt.rcParams['axes.labelsize']  = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

In [19]:
# Input parameters:
L                   = 4       #linear size of the system
T                   = 2.269   #a temperature for which there are MC configurations stored in Data_ising2d/MC_results
num_visible         = L*L     #number of visible nodes
num_hidden          = 4       #number of hidden nodes
nsteps              = 30000   #number of training steps (iterations over the mini-batches)
learning_rate_start = 1e-3    #the learning rate will start at this value and decay exponentially
bsize               = 100     #batch size
num_gibbs           = 10      #number of Gibbs iterations (steps of contrastive divergence)
num_samples         = 10      #number of chains in PCD

In [4]:
# Function to save weights and biases to a parameter file ###
def save_parameters(sess, rbm):
    weights, visible_bias, hidden_bias = sess.run([rbm.weights, rbm.visible_bias, rbm.hidden_bias])
    
    parameter_dir = 'Data_ising2d/RBM_parameters'
    if not(os.path.isdir(parameter_dir)):
      os.mkdir(parameter_dir)
    parameter_file_path =  '%s/parameters_nH%d_L%d' %(parameter_dir,num_hidden,L)
    parameter_file_path += '_T' + str(T)
    np.savez_compressed(parameter_file_path, weights=weights, visible_bias=visible_bias, hidden_bias=hidden_bias)

class Placeholders(object):
    pass

class Ops(object):
    pass

weights      = None  #weights
visible_bias = None  #visible bias
hidden_bias  = None  #hidden bias

In [11]:
# Load the MC configuration training data:
trainFileName = 'Data_ising2d/MC_results/ising2d_L'+str(L)+'_T'+str(T)+'_train.txt'
xtrain        = np.loadtxt(trainFileName)
testFileName  = 'Data_ising2d/MC_results/ising2d_L'+str(L)+'_T'+str(T)+'_test.txt'
xtest         = np.loadtxt(testFileName)

xtrain_randomized = np.random.permutation(xtrain) # random permutation of training data
xtest_randomized  = np.random.permutation(xtest) # random permutation of test data
iterations_per_epoch = xtrain.shape[0] / bsize

In [12]:
xtrain.shape

(20000, 16)

In [6]:
# Initialize the RBM class
rbm = RBM(num_hidden=num_hidden, num_visible=num_visible, weights=weights, visible_bias=visible_bias,hidden_bias=hidden_bias, num_samples=num_samples) 

# Initialize operations and placeholders classes
ops          = Ops()
placeholders = Placeholders()
placeholders.visible_samples = tf.placeholder(tf.float32, shape=(None, num_visible), name='v') # placeholder for training data

total_iterations = 0 # starts at zero 
ops.global_step  = tf.Variable(total_iterations, name='global_step_count', trainable=False)
learning_rate    = tf.train.exponential_decay(
    learning_rate_start,
    ops.global_step,
    100 * xtrain.shape[0]/bsize,
    1.0 # decay rate = 1 means no decay
)
  
cost      = rbm.neg_log_likelihood_forGrad(placeholders.visible_samples, num_gibbs=num_gibbs)
optimizer = tf.train.AdamOptimizer(learning_rate, epsilon=1e-2)
ops.lr    = learning_rate
ops.train = optimizer.minimize(cost, global_step=ops.global_step)
ops.init  = tf.group(tf.initialize_all_variables(), tf.initialize_local_variables())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.local_variables_initializer` instead.


In [13]:
# Define the negative log-likelihood
# We can use this to plot the RBM's training progress.
# This calculation is intractable for large networks so let's only do it for small num_hidden
logZ = rbm.exact_log_partition_function()
placeholders.logZ = tf.placeholder(tf.float32)
NLL = rbm.neg_log_likelihood(placeholders.visible_samples,placeholders.logZ)

sess = tf.Session()
sess.run(ops.init)
  
bcount         = 0  #counter
epochs_done    = 0  #epochs counter
nll_test_list  = [] #negative log-likelihood for each epoch
nll_train_list = [] #negative log-likelihood for each epoch
for ii in range(nsteps):
    if bcount*bsize+ bsize>=xtrain.shape[0]:
        bcount = 0
        xtrain_randomized = np.random.permutation(xtrain)

    batch     =  xtrain_randomized[ bcount*bsize: bcount*bsize+ bsize,:]
    bcount    += 1
    feed_dict =  {placeholders.visible_samples: batch}

    _, num_steps = sess.run([ops.train, ops.global_step], feed_dict=feed_dict)

    if num_steps % iterations_per_epoch == 0:
        lz = sess.run(logZ)
        nll_test = sess.run(NLL,feed_dict={placeholders.visible_samples: xtest_randomized, placeholders.logZ: lz})
        nll_test_list.append(nll_test)
    
        print ('Epoch = %d, Nsteps = %d, NLL on test data = %.6f' %(epochs_done,num_steps,nll_test))
        save_parameters(sess, rbm)
        epochs_done += 1

        ## Update the plot:
        #plt.figure(1)
        #plt.clf()
        #plt.plot( np.arange(epochs_done), nll_test_list, 'o-')
        #plt.xlabel('Epoch')
        #plt.ylabel('NLL')
        #plt.pause(0.1)

#plt.savefig('NLL_vs_epoch_T%s.pdf' %(str(T)))


Epoch = 0, Nsteps = 200, NLL on test data = 11.042135
Epoch = 1, Nsteps = 400, NLL on test data = 10.859882
Epoch = 2, Nsteps = 600, NLL on test data = 10.559417
Epoch = 3, Nsteps = 800, NLL on test data = 10.297823
Epoch = 4, Nsteps = 1000, NLL on test data = 10.047187
Epoch = 5, Nsteps = 1200, NLL on test data = 9.783507
Epoch = 6, Nsteps = 1400, NLL on test data = 9.499597
Epoch = 7, Nsteps = 1600, NLL on test data = 9.197001
Epoch = 8, Nsteps = 1800, NLL on test data = 8.872510
Epoch = 9, Nsteps = 2000, NLL on test data = 8.536477
Epoch = 10, Nsteps = 2200, NLL on test data = 8.191921
Epoch = 11, Nsteps = 2400, NLL on test data = 7.833353
Epoch = 12, Nsteps = 2600, NLL on test data = 7.473599
Epoch = 13, Nsteps = 2800, NLL on test data = 7.116506
Epoch = 14, Nsteps = 3000, NLL on test data = 6.767595
Epoch = 15, Nsteps = 3200, NLL on test data = 6.429122
Epoch = 16, Nsteps = 3400, NLL on test data = 6.099010
Epoch = 17, Nsteps = 3600, NLL on test data = 5.792073
Epoch = 18, Nsteps 

Then we can now sample spin configurations from the trained RBM

In [14]:
#Temperature list for which there are trained RBM parameters stored in Data_ising2d/RBM_parameters
#T_list = [1.0,1.254,1.508,1.762,2.016,2.269,2.524,2.778,3.032,3.286,3.540]
T_list = [2.269]

In [15]:
#Sampling parameters:
num_samples  = 500  # how many independent chains will be sampled
gibb_updates = 2    # how many gibbs updates per call to the gibbs sampler
nbins        = 100  # number of calls to the RBM sampler

In [16]:
#Specify where the sampled configurations will be stored:
samples_dir = 'Data_ising2d/RBM_samples'
if not(os.path.isdir(samples_dir)):
  os.mkdir(samples_dir)
samples_filePaths = [] #file paths where samples for each T will be stored

1.0

In [17]:
#Initialize the RBM for each temperature in T_list:
rbms           = []
rbm_samples    = []
for i in range(len(T_list)):
  T = T_list[i]
  
  samples_filePath =  '%s/samples_nH%d_L%d' %(samples_dir,num_hidden,L)
  samples_filePath += '_T' + str(T) + '.txt'
  samples_filePaths.append(samples_filePath)
  fout = open(samples_filePath,'w')
  fout.close()
  
  #Read in the trained RBM parameters:
  path_to_params =  'Data_ising2d/RBM_parameters/parameters_nH%d_L%d' %(num_hidden,L)
  path_to_params += '_T'+str(T)+'.npz'
  params         =  np.load(path_to_params)
  weights        =  params['weights']
  visible_bias   =  params['visible_bias']
  hidden_bias    =  params['hidden_bias']
  hidden_bias    =  np.reshape(hidden_bias,(hidden_bias.shape[0],1))
  visible_bias   =  np.reshape(visible_bias,(visible_bias.shape[0],1))
  
  # Initialize RBM class
  rbms.append(RBM(
    num_hidden=num_hidden, num_visible=num_visible,
    weights=weights, visible_bias=visible_bias,hidden_bias=hidden_bias,
    num_samples=num_samples
  ))
  rbm_samples.append(rbms[i].stochastic_maximum_likelihood(gibb_updates))
#end of loop over temperatures


In [18]:
# Initialize tensorflow
init = tf.group(tf.initialize_all_variables(), tf.initialize_local_variables())

# Sample thermodynamic observables:
N = num_visible
with tf.Session() as sess:
  sess.run(init)
  
  for i in range(nbins):
    print ('bin %d' %i)

    for t in range(len(T_list)):
      fout = open(samples_filePaths[t],'a')
      
      _,samples=sess.run(rbm_samples[t])
      spins = np.asarray((2*samples-1)) #convert from 0,1 variables to -1,+1 variables
      for k in range(num_samples):
        for i in range(N):
         fout.write('%d ' %int(spins[k,i]))
        fout.write('\n')
      fout.close()


bin 0
bin 1
bin 2
bin 3
bin 4
bin 5
bin 6
bin 7
bin 8
bin 9
bin 10
bin 11
bin 12
bin 13
bin 14
bin 15
bin 16
bin 17
bin 18
bin 19
bin 20
bin 21
bin 22
bin 23
bin 24
bin 25
bin 26
bin 27
bin 28
bin 29
bin 30
bin 31
bin 32
bin 33
bin 34
bin 35
bin 36
bin 37
bin 38
bin 39
bin 40
bin 41
bin 42
bin 43
bin 44
bin 45
bin 46
bin 47
bin 48
bin 49
bin 50
bin 51
bin 52
bin 53
bin 54
bin 55
bin 56
bin 57
bin 58
bin 59
bin 60
bin 61
bin 62
bin 63
bin 64
bin 65
bin 66
bin 67
bin 68
bin 69
bin 70
bin 71
bin 72
bin 73
bin 74
bin 75
bin 76
bin 77
bin 78
bin 79
bin 80
bin 81
bin 82
bin 83
bin 84
bin 85
bin 86
bin 87
bin 88
bin 89
bin 90
bin 91
bin 92
bin 93
bin 94
bin 95
bin 96
bin 97
bin 98
bin 99


# Read in the new states generated by RBM, calculate the physical observables 

In [20]:
sampleFileName = 'Data_ising2d/RBM_samples/samples_nH%d_L%d' %(num_hidden,L)
sampleFileName += '_T' + str(T) + '.txt'
xsample        = np.loadtxt(sampleFileName)


In [ ]:
# Function that calculate energy of the given state of 2D Ising model
def neighbor(site):
    
def energy(state):
    